In [8]:
import os
import cv2
import pandas as pd

folder_path = "Path"

# Define the function to split the video into segments
def split_video(video_path, num_segments=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Couldn't open the video file.")
        return []
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    if duration/num_segments >6:
        num_segments=5
    
    if duration/num_segments <6:
        num_segments=4

        if duration/num_segments <6:
            num_segments=3

            if duration/num_segments <6:
                num_segments=2


    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frames per segment
    frames_per_segment = total_frames // num_segments

    segments = []
    for i in range(num_segments):
        start_frame = i * frames_per_segment
        end_frame = start_frame + frames_per_segment

        # Append start and end frames to the list
        segments.append((start_frame, end_frame))

    cap.release()
    return segments, num_segments



def split_csv(csv_path, num_segments=5, output_folder=None):
    # Read the CSV file
    df = pd.read_csv(csv_path, header=None)

    # Determine the number of rows per segment
    total_rows = df.shape[1]
    rows_per_segment = total_rows // num_segments

    segments = []
    for i in range(num_segments):
        start_col = i * rows_per_segment
        end_col = (i + 1) * rows_per_segment

        # If it's the last segment, include remaining columns
        if i == num_segments - 1:
            end_col = total_rows

        # Append start and end columns to the list
        segments.append((start_col, end_col))

        # If output_folder is provided, save the segment to a new CSV file
        if output_folder:
            segment_df = df.iloc[:, start_col:end_col]
            segment_csv_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(csv_path))[0]}_segment_{i+1}.csv")
            segment_df.to_csv(segment_csv_path, index=False, header=False)

    return segments

# Get list of files in the folder
video_files = os.listdir(folder_path)
# Iterate through each file
for video_file in video_files:
    # Check if the file is a video file (assuming extension is ".mp4")
    if not video_file.endswith(".mp4"):
        # If it's not an MP4 file, skip to the next iteration
        continue
    
    # Construct paths for video and corresponding CSV file
    video_path = os.path.join(folder_path, video_file)
    csv_file = os.path.splitext(video_file)[0] + ".csv"
    csv_path = os.path.join(folder_path, csv_file)
    
    # Create output folders for video segments and CSV segments
    video_output_folder = os.path.join(folder_path, "video_segments")
    csv_output_folder = os.path.join(folder_path, "csv_segments")
    os.makedirs(video_output_folder, exist_ok=True)
    os.makedirs(csv_output_folder, exist_ok=True)
    
    # Split the video into segments
    segments , num_segments= split_video(video_path, num_segments=5)
    
    for segment_idx, (start_frame, end_frame) in enumerate(segments):
        # Process each segment separately
        
        # Split the CSV data into segments
        csv_segments = split_csv(csv_path, num_segments=num_segments, output_folder=csv_output_folder)
        start_col, end_col = csv_segments[segment_idx]
        
        # Read relevant data for this segment from CSV
        segment_data = pd.read_csv(csv_path, usecols=range(start_col, end_col), header=None)
        Y1 = segment_data.iloc[0]
        Y2 = segment_data.iloc[1]
        
        # Extract frames for this segment
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        segment_frames = []
        while cap.isOpened() and cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame:
            ret, frame = cap.read()
            if not ret:
                break
            segment_frames.append(frame)
        cap.release()
        
        # Save the segment frames to a new video file
        segment_video_path = os.path.join(video_output_folder, f"{os.path.splitext(video_file)[0]}_segment_{segment_idx+1}.mp4")
        out = cv2.VideoWriter(segment_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (segment_frames[0].shape[1], segment_frames[0].shape[0]))
        for frame in segment_frames:
            out.write(frame)
        out.release()
        
        # Print a message indicating which segment is being processed
        print(f"Processed segment {segment_idx+1}/{len(segments)} of video: {video_path} and CSV: {csv_path}")

print("Processed all segment")

Processed segment 1/5 of video: /Users/janikmori/Desktop/Test_videos/O7.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O7.csv
Processed segment 2/5 of video: /Users/janikmori/Desktop/Test_videos/O7.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O7.csv
Processed segment 3/5 of video: /Users/janikmori/Desktop/Test_videos/O7.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O7.csv
Processed segment 4/5 of video: /Users/janikmori/Desktop/Test_videos/O7.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O7.csv
Processed segment 5/5 of video: /Users/janikmori/Desktop/Test_videos/O7.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O7.csv
Processed segment 1/5 of video: /Users/janikmori/Desktop/Test_videos/O6.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O6.csv
Processed segment 2/5 of video: /Users/janikmori/Desktop/Test_videos/O6.mp4 and CSV: /Users/janikmori/Desktop/Test_videos/O6.csv
Processed segment 3/5 of video: /Users/janikmori/Desktop/Test_videos/O6.mp4 and CSV: /Users/janik